<a href="https://colab.research.google.com/github/emasahisa908-cmyk/NLP-Week2-Text-Generation/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

print("✔ Google Drive mounted.")

Mounted at /content/drive
✔ Google Drive mounted.


In [ ]:
# import library
import pandas as pd

In [ ]:
#path for dataset
recipe_path = "/content/drive/MyDrive/RecipeNLG_dataset.csv"

In [ ]:

#load data
print("Loading dataset...")
df = pd.read_csv(recipe_path)
print("all recipes:", len(df))

Loading dataset...
all recipes: 2231142


In [ ]:

# keyward for dessert
dessert_keywords = [
    "cake", "cookie", "brownie", "pie", "pudding",
    "ice cream", "tart", "sweet", "dessert",
    "muffin", "cupcake", "sorbet", "candy",
    "chocolate", "custard", "donut", "cheesecake",
    "bread pudding", "waffle", "pancake"
]

In [ ]:
#Dessert Determination Function
def is_dessert(text):
    text = str(text).lower()
    return any(keyword in text for keyword in dessert_keywords)

In [ ]:

# extract dessert recipes
print("Extracting dessert recipes...")

desserts_df = df[
    df["title"].apply(is_dessert) |
    df["ingredients"].apply(is_dessert) |
    df["directions"].apply(is_dessert)
]

print("Number of extracted desserts:", len(desserts_df))

Extracting dessert recipes...
Number of extracted desserts: 1085463


In [ ]:
# make storage folder for only dessert recipes
import os
os.makedirs("/content/drive/MyDrive/recipeNLG", exist_ok=True)

# store
save_path = "/content/drive/MyDrive/recipeNLG/desserts_only.csv"
desserts_df.to_csv(save_path, index=False)

print("✔ Dessert recipe saved →", save_path)


✔ Dessert recipe saved → /content/drive/MyDrive/recipeNLG/desserts_only.csv


In [ ]:
# import library
!pip3 install transformers sentence-transformers faiss-cpu pandas numpy -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 112.2 MB/s eta 0:00:00


In [ ]:
#load library
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import json
import faiss

In [ ]:
# load dessert only CVS
df = pd.read_csv("/content/drive/MyDrive/recipeNLG/desserts_only.csv")

print("Loaded dessert recipes:", len(df))

Loaded dessert recipes: 1085463


In [ ]:
# ---------------------------
# 5. Build RAG records
# ---------------------------
records = []
for idx, row in df.iterrows():
    full_text = f"{row['title']}\nIngredients: {row['ingredients']}\nInstructions: {row['directions']}"
    records.append({
        "id": idx,
        "title": row["title"],
        "ingredients": row["ingredients"],
        "instructions": row["directions"],
        "text": full_text
    })

print("Sample record:", records[0])

Sample record: {'id': 0, 'title': 'No-Bake Nut Cookies', 'ingredients': '["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]', 'instructions': '["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]', 'text': 'No-Bake Nut Cookies\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\nInstructions: ["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium

In [ ]:

# ---------------------------
# 6. Embedding + FAISS index
# ---------------------------
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

recipe_texts = [r["text"] for r in records]
recipe_embeddings = embedding_model.encode(recipe_texts, convert_to_numpy=True)

print("Embeddings:", recipe_embeddings.shape)

dimension = recipe_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(recipe_embeddings)

print("FAISS index size:", index.ntotal)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings: (1085463, 384)
FAISS index size: 1085463


In [ ]:

# ====================================================
# 7. Retriever
# ====================================================
def retrieve_recipes(user_input, k=5):
    query_vec = embedding_model.encode([user_input], convert_to_numpy=True)
    distances, ids = index.search(query_vec, k)
    results = [records[i] for i in ids[0]]
    return results

In [ ]:
# ====================================================
# 8. Load LLM (Mistral-7B-Instruct)
# ====================================================
from transformers import AutoTokenizer, pipeline

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

llm = pipeline(
    "text-generation",
    model=MODEL_ID,
    tokenizer=tokenizer,
    torch_dtype="auto",
    device_map="auto",
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# ====================================================
# 9. Generator (Final unified version)
# ====================================================
def generate_with_rag(user_ingredients: str, k: int = 3, max_new_tokens: int = 600):

    # 1) Retrieve top-k recipes
    retrieved = retrieve_recipes(user_ingredients, k=k)

    # 2) Build SHORT context
    context_blocks = []
    for r in retrieved:
        ing_short = str(r["ingredients"])[:120] + "..."
        block = f"Title: {r['title']}\nMain ingredients: {ing_short}"
        context_blocks.append(block)

    context_text = "\n\n---\n\n".join(context_blocks)

    # 3) Strong instruction to avoid hallucinations
    prompt = f"""
You are a dessert recipe recommendation assistant.

TASK:
- Recommend EXACTLY 1–2 desserts the user can make.
- Use ONLY the recipe titles and instructions from the context.
- For each dessert:
   1) Give the dessert name
   2) Give a short explanation why it matches the user's ingredients
   3) The method MUST summarize the instructions from the context, not generic baking steps.
   4) Give a 1–2 sentence summary of the cooking method based on the context
- DO NOT repeat the full ingredient list.
- DO NOT output extra text.
- Follow this format:

1. <Dessert name> — <why it matches>
   Method: <short summary>

2. <Dessert name> — <why it matches>
   Method: <short summary>

User ingredients:
{user_ingredients}

Recipe context:
{context_text}

Answer:
"""

    output = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
    )

    generated = output[0]["generated_text"]

    if "Answer:" in generated:
        generated = generated.split("Answer:", 1)[1].strip()

    return generated

In [ ]:

# ====================================================
# 10. Test
# ====================================================
test_query = "flour sugar eggs butter cocoa powder"
print("💡 Test ingredients:", test_query)
print("\n🧁 SweetFinder suggestion:\n")
print(generate_with_rag(test_query))

💡 Test ingredients: flour sugar eggs butter cocoa powder

🧁 SweetFinder suggestion:



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. Cocoa-Buttermilk Birthday Cake — This recipe uses flour, sugar, eggs, butter, cocoa powder, which are all included in the user's ingredients.
   Method: In a mixing bowl, combine dry ingredients (2 cups all-purpose flour, 1/2 cup unsweetened cocoa powder, 1/2 teaspoon baking powder, 1/2 teaspoon baking soda). In a separate bowl, whisk together wet ingredients (1 cup buttermilk, 1 teaspoon vanilla extract, 1/2 cup vegetable oil). Pour wet ingredients into the dry ingredients and mix until just combined. Pour batter into a greased 9-inch cake pan and bake at 350°F for 30-35 minutes or until a toothpick inserted into the center comes out clean.

2. Dark Cocoa Buttermilk Cake With Chocolate Cream Cheese Frosting — This recipe also uses flour, sugar, eggs, butter, and cocoa powder from the user's ingredients.
   Method: Preheat oven to 375°F. In a mixing bowl, cream together butter and sugar until light and fluffy. Add eggs one at a time, beating well after each addition. In a separate b

In [ ]:

# ====================================================
# 11. Gradio UI
# ====================================================
import gradio as gr

def sweetfinder_interface(user_ingredients):
    if not user_ingredients.strip():
        return "Please enter at least one ingredient."
    try:
        return generate_with_rag(user_ingredients)
    except Exception as e:
        return f"Error: {str(e)}"

with gr.Blocks(title="SweetFinder – Dessert Recommendation AI") as demo:

    gr.Markdown("""
    # 🍰 SweetFinder
    **A dessert recommendation assistant powered by a RAG pipeline.**
    Enter ingredients you have → receive desserts you can make.
    """)

    input_box = gr.Textbox(
        label="Ingredients",
        placeholder="Example: flour, sugar, eggs, butter, chocolate"
    )

    output_box = gr.Textbox(
        label="Recommended Desserts",
        lines=12
    )

    submit_btn = gr.Button("Find Desserts 🍪")
    submit_btn.click(sweetfinder_interface, inputs=input_box, outputs=output_box)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5c7e23e7b8bfe771c5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
